In [16]:
import lxml
import requests
#from amazon_review_scraper import amazon_review_scraper
import os
import csv
from lxml import html  
#from exceptions import ValueError
import time
from time import sleep
import pandas as pd
import numpy as np
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup as BS
import ssl
import json
from pandas.io.json import json_normalize
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import csv
import pandas as pd
import math
from math import ceil
from urllib.request import Request,  urlopen    # for opening links collected
from urllib.error import HTTPError     # exception handling for errors etc
import re    # for regex
import timeit    # for runtime measurement
import nltk
from textblob import TextBlob
from sklearn.decomposition import LatentDirichletAllocation,PCA
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import re
import os.path
import os
import io
# Gettnig rid of unnecessary warnings
import warnings; 
warnings.simplefilter('ignore')


In [5]:

def get_browser():

    locationOfWebdriver = "C:\BhargaviM\CBA\Term1 R2\Practicum Term1\chromedriver.exe"
    browser = webdriver.Chrome(locationOfWebdriver)
    GlobalBrowser = browser
    return browser

GlobalBrowser = get_browser()

def get_apple_asins():

    url="https://www.amazon.com/"

    GlobalBrowser.get(url) #Opening the URL
    time.sleep(10) #Waiting for the page to load
    search_element=GlobalBrowser.find_element_by_id("twotabsearchtextbox")
    search_element.send_keys("iPhone")

    try:
        search_button=GlobalBrowser.find_element_by_xpath('//*[@id="nav-search"]/form/div[2]/div/input')
    except:
        pass
    #Clicking the search button
    try:
        search_button.click()
    except:
        pass
    time.sleep(5)

    try:
        select_apple=GlobalBrowser.find_element_by_xpath('//*[@id="leftNavContainer"]/ul[3]/div/li[1]/span/span/div/label/span/span')
    except:
        pass
    #Clicking the search button
    try:
        select_apple.click()
    except:
        pass
    time.sleep(5)

    
    amaz_search_url = GlobalBrowser.current_url
    print("APPLE URL : " + amaz_search_url)

    GlobalBrowser.get(amaz_search_url)
    source = GlobalBrowser.page_source
    soup = BS(source, 'html.parser')
    number_of_search_results = soup.find('span', {'id': 's-result-count'})
    non_sponsered_results = soup.find_all(id=re.compile("result_"))

    asins = []
    for result in non_sponsered_results:
        asins.append(result['data-asin'])

    unique_asins = list(set(asins))
    print("Fetched APPLE ASINS : ")
    print(unique_asins)
    return unique_asins


def get_oneplus_asins():

    url="https://www.amazon.com/"

    GlobalBrowser.get(url) #Opening the URL
    time.sleep(10) #Waiting for the page to load
    search_element=GlobalBrowser.find_element_by_id("twotabsearchtextbox")
    search_element.send_keys("oneplus")

    try:
        search_button=GlobalBrowser.find_element_by_xpath('//*[@id="nav-search"]/form/div[2]/div/input')
    except:
        pass
    #Clicking the search button
    try:
        search_button.click()
    except:
        pass
    time.sleep(5)

    try:
        select_oneplus=GlobalBrowser.find_element_by_xpath('//*[@id="leftNavContainer"]/ul[2]/div/li[1]/span/span/div/label/span/span')

    except:
        pass
    #Clicking the search button
    try:
        select_oneplus.click()
    except:
        pass
    time.sleep(5)

    
    amaz_search_url = GlobalBrowser.current_url
    print("ONEPLUS URL : " + amaz_search_url)

    GlobalBrowser.get(amaz_search_url)
    source = GlobalBrowser.page_source
    soup = BS(source, 'html.parser')
    number_of_search_results = soup.find('span', {'id': 's-result-count'})
    non_sponsered_results = soup.find_all(id=re.compile("result_"))

    asins = []
    for result in non_sponsered_results:
        asins.append(result['data-asin'])

    unique_asins = list(set(asins))
    print("Fetched ONEPLUS ASINS : ")
    print(unique_asins)
    return unique_asins

def get_positive_review_count(soup):
    count = '0'   
    if soup.find('a', {'data-reftag': 'cm_cr_arp_d_viewpnt_lft'}):
        count = soup.find('a', {'data-reftag': 'cm_cr_arp_d_viewpnt_lft'})
        if count.text:
            count = re.sub(r'\D','',count.text)

    if count:
        return count
    else:
        count = '0'
        return count


def get_critical_review_count(soup):
    count = '0'
    if soup.find('a', {'data-reftag': 'cm_cr_arp_d_viewpnt_rgt'}):
        count = soup.find('a', {'data-reftag': 'cm_cr_arp_d_viewpnt_rgt'})
        if count.text:
            count = re.sub(r'\D','',count.text)

    if count:
        return count
    else:
        count = '0'
        return count

def get_review_title(soup):
    review_title = 'No Review Title Found'
    if soup.find('a',{'data-hook': 'review-title'}):
        review_title = soup.find('a',{'data-hook': 'review-title'})
        if review_title.text:
            review_title = str(review_title.text)
    return review_title


def get_review_stars(soup):
    review_ratings = 'No Rating'
    if soup.find('span', attrs={'class': 'a-icon-alt'}):
        star = soup.find('span', attrs={'class': 'a-icon-alt'})
        if star.text:
            review_ratings = star.text.split()[0]            
    return review_ratings

def get_review_text(soup):
    review_text = 'No text'
    if soup.find('span', {'data-hook': 'review-body'}):
        review_text = soup.find('span', {'data-hook': 'review-body'})
        if review_text.text:
            review_text = review_text.text.replace('\U0001f44d', '').replace('\U0001f4a9', '')
    return review_text


def get_reviewer_name(soup):
    reviewer_name = ''
    if soup.find('span',{'class': 'a-profile-name'}):
        reviewer_name = soup.find('span',{'class': 'a-profile-name'})
        if reviewer_name.text:
            reviewer_name = str(reviewer_name.text)
    return reviewer_name


def get_review_date(soup):
    review_date = ''
    if soup.find('span', {'data-hook': 'review-date'}):
        review_date = soup.find('span', {'data-hook': 'review-date'})
        if review_date.text:
            review_date = str(review_date.text)
    return review_date


def get_helpful_comment(soup):
    review_helpful = 'Not Available'
    if soup.find('span', {'data-hook': 'helpful-vote-statement'}):
        review_helpful = soup.find('span', {'data-hook': 'helpful-vote-statement'})
        if review_helpful.text:
            review_helpful = str(review_helpful.text)
            review_helpful = review_helpful.split()[0]
    return review_helpful


def get_review_link(soup):
    review_link = ''
    links = soup.find_all('a',{'class': 'a-size-base a-link-normal review-title a-color-base a-text-bold'}, href=True)
    links = ["https://www.amazon.com%s" % l['href'] for l in links]
    for ll in links:
        review_link = ll
    return review_link





def get_positive_reviews(browser,url,total_positive_review_count,lst):

    positive_review_count = total_positive_review_count

    if positive_review_count:
        positive_review_count = int(positive_review_count)
    page_count = int(math.ceil(positive_review_count/10))
    page_count = 5

    try:
        GlobalBrowser.get(url + f'/ref=cm_cr_arp_d_viewpnt_lft?pageNumber=1&filterByStar=positive')
    except:
        pass


    if page_count > 0:
        print(f'Page count: {str(page_count)}')
        for i in range(page_count):
            page = i + 1
            page = str(page)
            print(f'Fetching page {page}')
            try:
                GlobalBrowser.find_element_by_link_text(page).click()
            except:
                pass
            time.sleep(10)
            html = GlobalBrowser.page_source
            paged_soup = BS(html, 'html.parser')

            for review_sections in paged_soup.find_all('div', {'data-hook': 'review'}):
                product_lines.append(lst[0])
                product_asins.append(lst[1])
                product_titles.append(lst[2])
                overall_ratings.append(lst[3])
                total_product_reviews.append(lst[4])
                total_positive_reviews.append(lst[5])
                total_critical_reviews.append(lst[6])
                reviewer_names.append(get_reviewer_name(review_sections))
                review_titles.append(get_review_title(review_sections))
                review_dates.append(get_review_date(review_sections))
                review_links.append(get_review_link(review_sections))
                review_ratings.append(get_review_stars(review_sections))
                helpful_count.append(get_helpful_comment(review_sections))
                review_texts.append(get_review_text(review_sections))
                review_type.append("Positive")


def get_critical_reviews(browser,url,total_critical_review_count,lst):

    critical_review_count = total_critical_review_count
    if critical_review_count:
        critical_review_count = int(critical_review_count)
    page_count = int(math.ceil(critical_review_count/10))
    page_count = 5

    try:
        GlobalBrowser.get(url + f'/ref=cm_cr_arp_d_viewpnt_lft?pageNumber=1&filterByStar=critical')
    except:
        pass

    if page_count > 0:
        print(f'Page count: {str(page_count)}')
        for i in range(page_count):
            page = i + 1
            page = str(page)
            print(f'Fetching page {page}')
            try:
                GlobalBrowser.find_element_by_link_text(page).click()
            except:
                pass
            time.sleep(10)
            html = GlobalBrowser.page_source
            paged_soup = BS(html, 'html.parser')

            for review_sections in paged_soup.find_all('div', {'data-hook': 'review'}):
                product_lines.append(lst[0])
                product_asins.append(lst[1])
                product_titles.append(lst[2])
                overall_ratings.append(lst[3])
                total_product_reviews.append(lst[4])
                total_positive_reviews.append(lst[5])
                total_critical_reviews.append(lst[6])
                reviewer_names.append(get_reviewer_name(review_sections))
                review_titles.append(get_review_title(review_sections))
                review_dates.append(get_review_date(review_sections))
                review_links.append(get_review_link(review_sections))
                review_ratings.append(get_review_stars(review_sections))
                helpful_count.append(get_helpful_comment(review_sections))
                review_texts.append(get_review_text(review_sections))
                review_type.append("Critical")


def read_reviews(GlobalBrowser,asins,product_line):

    base_url = 'https://www.amazon.com/product-reviews/'

    asins = asins

    print("========================> Fetching reviews for " + product_line)

    if len(asins) > 0:
        for asin in asins:
            
            print("Fetching reviews for ASIN :" + asin)
            # get reviews page count
            url = base_url + asin
            GlobalBrowser.get(url)
            source = GlobalBrowser.page_source

            soup = BS(source, 'html.parser')
            # soup = soup.encode("utf-8")

            total_reviews = soup.find('span', {'data-hook': 'total-review-count'})
            total_reviews = int(total_reviews.text.replace(",",""))

            #3.9 out of 5 stars
            total_review_rating = soup.find('span',{'data-hook':'rating-out-of-text'})
            total_review_rating = (total_review_rating.text).split()
            total_review_rating = total_review_rating[0]
            

            total_reviews_count = total_reviews


            #wont consider asins whose total reviews are less then 10
            if total_reviews_count < 10:
                print("Ignoring ASIN : " + asin + " as there are no reviews")
                continue


            total_positive_review_count = get_positive_review_count(soup)
            total_critical_review_count = get_critical_review_count(soup)
            total_overall_review_rating = total_review_rating
            print("ASIN overall rating : " + total_overall_review_rating)
            print("ASIN positive rating : " + total_positive_review_count)
            print("ASIN critical rating : " + total_critical_review_count)



            # grab the title
            product_title = ""
            if soup.find('a', {'data-hook': 'product-link'}):
                product_title = soup.find('a', {'data-hook': 'product-link'})
                if product_title.text:
                    product_title = str(product_title.text)
                else:
                    product_title = 'No title found'
            else:
                product_title = 'No title found'

            print("ASIN product name : " + product_title)

            
            lst = []
            lst.append(product_line)
            lst.append(asin)
            lst.append(product_title)
            lst.append(total_overall_review_rating)
            lst.append(total_reviews)
            lst.append(total_positive_review_count)
            lst.append(total_critical_review_count)


            get_positive_reviews(GlobalBrowser,url,total_positive_review_count,lst)
            get_critical_reviews(GlobalBrowser,url,total_critical_review_count,lst)
       
       


product_lines = []
product_asins = []
product_titles = []
overall_ratings = []
total_product_reviews = []
review_type = []
total_positive_reviews = []
total_critical_reviews = []
reviewer_names = []
review_titles = []
review_dates = []
review_links = []
review_ratings = []
helpful_count = []
review_texts = []




def mainFunc():
    GlobalBrowser = get_browser()
    apple_asins = get_apple_asins()
    oneplus_asins = get_oneplus_asins()
    product_line = "APPLE"
    #apple_asins=['B075QN8NDH']
    #oneplus_asins=['B06Y3NQBZM']
    read_reviews(GlobalBrowser,apple_asins,product_line)  
    product_line = "ONEPLUS"
    read_reviews(GlobalBrowser,oneplus_asins,product_line)
    print(product_lines)
    print(product_asins)
    print(product_titles)
    print(overall_ratings)
    print(total_product_reviews)
    print(review_type)
    print(total_positive_reviews)
    print(total_critical_reviews)
    print(reviewer_names)
    print(review_titles)
    print(review_dates)
    print(review_links)
    print(review_ratings)
    print(helpful_count)
    print(review_texts)

    GlobalBrowser.close()

    

mainFunc()


APPLE URL : https://www.amazon.com/s/s/ref=sr_nr_p_89_0?fst=as%3Aoff&rh=n%3A2335752011%2Cn%3A7072561011%2Ck%3AiPhone%2Cp_89%3AApple&keywords=iPhone&ie=UTF8&qid=1545981135&rnid=2528832011
Fetched APPLE ASINS : 
['B01M1EXQY4', 'B07HCTC336', 'B01DAJS4LE', 'B075LVZSLW', 'B075X4DCW3', 'B07HKYQGD2', 'B078HVJB69', 'B07759W12Z', 'B07HD1M5HD', 'B07MGMG3WM', 'B071CS3NXJ', 'B01LXU4VO7', 'B07CZNHLN6', 'B07B9P995P', 'B01N2K14U7', 'B00VE1E0CI', 'B07K8K99TD', 'B075QMZH2L', 'B07K8L56DR', 'B07K97BQDF', 'B07HCZ6WGM', 'B07K8SY2S9', 'B01LWAM0V1', 'B00NQHZ0EA', 'B07HD3KF7V', 'B075QN8NDH', 'B07HCRG2VD', 'B075QNGHS8']
ONEPLUS URL : https://www.amazon.com/s/s/ref=sr_nr_p_89_0?fst=as%3Aoff&rh=i%3Aaps%2Ck%3Aoneplus%2Cp_89%3AOnePlus&keywords=oneplus&ie=UTF8&qid=1545981164&rnid=2528832011
Fetched ONEPLUS ASINS : 
['B07CK7KB95', 'B013WLT63A', 'B07D7Q9PPG', 'B017IOHGTI', 'B0789MN114', 'B0785H6VKX', 'B07DFYM9RC', 'B077TQN1BC', 'B0734X8GW5', 'B07D7BPQ26', 'B01N6NFCJ4', 'B01NBCSXXL', 'B07D5PJBRC', 'B06Y3NQBZM', 'B0778

Page count: 5
Fetching page 1
Fetching page 2
Fetching page 3
Fetching page 4
Fetching page 5
Page count: 5
Fetching page 1
Fetching page 2
Fetching page 3
Fetching page 4
Fetching page 5
========================> Fetching reviews for ONEPLUS
Fetching reviews for ASIN :B07CK7KB95
Ignoring ASIN : B07CK7KB95 as there are no reviews
Fetching reviews for ASIN :B013WLT63A
ASIN overall rating : 4.0
ASIN positive rating : 31
ASIN critical rating : 8
ASIN product name : Oneplus 2 Factory Unlocked A2003 4G Smartphone, Android 5.1, 5.5 Inch, Dual Nano-SIM Card - 64 GB Black (International version, No Warranty)
Page count: 5
Fetching page 1
Fetching page 2
Fetching page 3
Fetching page 4
Fetching page 5
Page count: 5
Fetching page 1
Fetching page 2
Fetching page 3
Fetching page 4
Fetching page 5
Fetching reviews for ASIN :B07D7Q9PPG
Ignoring ASIN : B07D7Q9PPG as there are no reviews
Fetching reviews for ASIN :B017IOHGTI
ASIN overall rating : 3.8
ASIN positive rating : 20
ASIN critical rating : 9


In [6]:
df= pd.DataFrame(columns=['product_asins','product_lines' ,'product_titles', 'overall_ratings', 'total_product_reviews', 'review_type', 'total_positive_reviews', 'total_critical_reviews', 'reviewer_names', 'review_titles','review_dates', 'review_links','review_ratings','helpful_count','review_texts'])

df.product_lines =product_lines
df.product_asins = product_asins
df.product_titles= product_titles
df.overall_ratings= overall_ratings
df.total_product_reviews=total_product_reviews
df.review_type =review_type
df.total_positive_reviews=total_positive_reviews
df.total_critical_reviews=total_critical_reviews
df.reviewer_names=reviewer_names
df.review_titles=review_titles
df.review_dates=review_dates
df.review_links=review_links
df.review_ratings=review_ratings
df.helpful_count=helpful_count
df.review_texts=review_texts
print(df)

     product_asins product_lines  \
0       B01M1EXQY4         APPLE   
1       B01M1EXQY4         APPLE   
2       B01M1EXQY4         APPLE   
3       B01M1EXQY4         APPLE   
4       B01M1EXQY4         APPLE   
5       B01M1EXQY4         APPLE   
6       B01M1EXQY4         APPLE   
7       B01M1EXQY4         APPLE   
8       B01M1EXQY4         APPLE   
9       B01M1EXQY4         APPLE   
10      B01M1EXQY4         APPLE   
11      B01M1EXQY4         APPLE   
12      B01M1EXQY4         APPLE   
13      B01M1EXQY4         APPLE   
14      B01M1EXQY4         APPLE   
15      B01M1EXQY4         APPLE   
16      B01M1EXQY4         APPLE   
17      B01M1EXQY4         APPLE   
18      B01M1EXQY4         APPLE   
19      B01M1EXQY4         APPLE   
20      B01M1EXQY4         APPLE   
21      B01M1EXQY4         APPLE   
22      B01M1EXQY4         APPLE   
23      B01M1EXQY4         APPLE   
24      B01M1EXQY4         APPLE   
25      B01M1EXQY4         APPLE   
26      B01M1EXQY4         A

In [7]:
res_csv = df.to_csv('C:\BhargaviM\CBA\Term1 R2\Practicum Term1\Amazon_reviews/output_amazon_rev_sample.csv', sep=',', encoding='utf-8')

In [39]:
polarity= []
polarity_values = []
priority_num = 0.0
def get_polarity(df):    
    for txt in df['review_texts']:
        priority_num = TextBlob(str(txt)).sentiment.polarity                
        print(priority_num)
        polarity.append(priority_num)
        if priority_num == 0.0:
            polarity_values.append('NEUTRAL')
        elif priority_num >= -1.0 and priority_num < 0.0:
            polarity_values.append('NEGATIVE')
        elif priority_num > 0.0 and priority_num <= 1.0:
            polarity_values.append('POSITIVE')


get_polarity(df)
df['polarity'] = polarity
df['polarity_value'] = polarity_values

0.39999999999999997
0.7
0.7
1.0
0.30833333333333335
0.0
0.7
0.2
1.0
0.3381640625
0.4787878787878788
0.7
0.6000000000000001
0.5625
1.0
0.16249999999999998
1.0
0.0
0.35
0.2
0.3371212121212121
0.25
0.39999999999999997
0.0
0.25
0.22499999999999998
0.75
0.3333333333333333
0.1619047619047619
0.9765625
0.5
0.5
0.2857142857142857
0.8
1.0
0.0
0.0
1.0
1.0
0.8
0.7
0.6333333333333333
0.27
0.29166666666666663
-0.14375
0.2522727272727273
0.25
0.42499999999999993
0.44999999999999996
0.7
-0.07098039215686275
0.0
0.13666666666666666
0.0
-1.0
0.0
-0.5
0.27499999999999997
0.0
-0.16666666666666666
0.0
0.11366666666666667
-0.5
1.0
0.0
0.0
0.0
-0.75
0.125
0.2833333333333333
0.13333333333333333
-0.25
0.16666666666666666
0.0
-0.09722222222222221
0.1519715791454922
-3.4343434343436786e-05
0.06271645021645021
-1.0
0.001776960784313729
0.03033459595959595
-0.24706439393939394
0.02835464015151515
-0.1666666666666667
0.275
-0.16249999999999998
0.031994047619047616
0.3
-0.78125
0.029720279720279713
-0.8
0.0
0.0
0.0

0.26
-0.09872159090909091
0.1431818181818182
0.4119318181818181
0.0
0.075
0.35
0.8
0.26
-0.09872159090909091
0.1431818181818182
0.4119318181818181
0.0
0.075
0.35
-0.027777777777777776
0.13636363636363635
0.7000000000000001
0.2875
0.25
0.2666666666666667
0.13636363636363635
0.0
0.4
1.0
0.35555555555555557
0.2931818181818182
0.3791666666666667
0.23333333333333336
0.4222222222222222
0.36818181818181817
0.0
0.3340909090909091
0.0
0.0
1.0
0.31666666666666665
0.8
0.25
-0.05416666666666667
0.75
0.2
0.09999999999999998
0.0
0.85
1.0
1.0
0.4
0.7
1.0
0.490625
0.75
0.0
0.5
0.7
0.5
0.7
0.0
0.35
0.14
0.5
1.0
0.8
1.0
0.8
-0.28484848484848485
-0.25
-0.5
0.13636363636363635
0.0
0.0
0.7
0.0
0.19166666666666665
0.0
0.0
-0.1
-0.09375000000000001
0.16523809523809518
0.2352864583333333
0.12677377960581088
0.33291903409090906
-0.07619047619047621
0.14357667584940315
-0.04403231448685994
0.15242753623188407
0.2
0.0
0.0
0.0
-0.125
0.2
0.4375
-0.16666666666666666
0.0
-0.22499999999999998
-0.525
-0.0374999999999

0.52
0.287037037037037
1.0
0.3933333333333333
0.21477272727272725
0.9
1.0
0.5
0.8
0.16666666666666669
0.4
1.0
0.7
0.9
0.5666666666666667
0.0
0.75
0.65
0.6
0.5078125
0.32223248106060604
0.3036111111111112
0.47878787878787876
0.05517202096149463
0.6166666666666666
0.2750877192982456
0.6428571428571428
0.5845703125
0.45555555555555555
0.3492857142857143
0.3263888888888889
0.75
0.0
0.294812925170068
1.0
0.30833333333333335
-0.16166666666666668
0.0
0.2
-0.2
0.0
0.0
0.5
0.2583333333333333
0.09405212842712844
-0.05000000000000001
-0.05000000000000001
-0.05000000000000001
-0.05000000000000001
0.3458333333333334
0.225
0.35000000000000003
0.5549999999999999
0.9333333333333332
0.5375
0.1791666666666667
1.0
0.5666666666666667
1.0
0.12307692307692308
1.0
0.9099999999999999
0.2006696428571429
0.33
0.375
0.0
0.4488636363636364
0.7
0.5549999999999999
0.2333333333333333
0.21098484848484844
-0.05
0.5
0.26666666666666666
0.5388888888888889
0.8
0.0
0.4083333333333333
0.32414965986394556
0.1846153846153846

0.020176767676767675
-0.05000000000000001
0.08792783274926133
0.2625
0.020176767676767675
-0.05000000000000001
0.08792783274926133
0.2625
0.033223090468853175
0.0
-0.016666666666666663
0.5666666666666667
0.033223090468853175
0.0
-0.016666666666666663
0.5666666666666667


In [40]:
print(polarity_df[:20])

   product_asins product_lines  \
0     B01M1EXQY4         APPLE   
1     B01M1EXQY4         APPLE   
2     B01M1EXQY4         APPLE   
3     B01M1EXQY4         APPLE   
4     B01M1EXQY4         APPLE   
5     B01M1EXQY4         APPLE   
6     B01M1EXQY4         APPLE   
7     B01M1EXQY4         APPLE   
8     B01M1EXQY4         APPLE   
9     B01M1EXQY4         APPLE   
10    B01M1EXQY4         APPLE   
11    B01M1EXQY4         APPLE   
12    B01M1EXQY4         APPLE   
13    B01M1EXQY4         APPLE   
14    B01M1EXQY4         APPLE   
15    B01M1EXQY4         APPLE   
16    B01M1EXQY4         APPLE   
17    B01M1EXQY4         APPLE   
18    B01M1EXQY4         APPLE   
19    B01M1EXQY4         APPLE   

                                      product_titles overall_ratings  \
0   Apple iPhone 7 128 GB Unlocked, Black US Version             3.6   
1   Apple iPhone 7 128 GB Unlocked, Black US Version             3.6   
2   Apple iPhone 7 128 GB Unlocked, Black US Version             3.6 

In [42]:
res_csv1 = df.to_csv('C:\BhargaviM\CBA\Term1 R2\Practicum Term1\Amazon_reviews/output_amazon_rev_polarity.csv', sep=',', encoding='utf-8')

In [43]:
shuffled_df = shuffle(df, random_state=42).reset_index(drop=True)
print(shuffled_df)

     product_asins product_lines  \
0       B071CS3NXJ         APPLE   
1       B07CZNHLN6         APPLE   
2       B013WLT63A       ONEPLUS   
3       B075X4DCW3         APPLE   
4       B0785H6VKX       ONEPLUS   
5       B00NQHZ0EA         APPLE   
6       B071CS3NXJ         APPLE   
7       B075X4DCW3         APPLE   
8       B01DAJS4LE         APPLE   
9       B01DAJS4LE         APPLE   
10      B01DAJS4LE         APPLE   
11      B01M1EXQY4         APPLE   
12      B07HD3KF7V         APPLE   
13      B01GCTAM8M       ONEPLUS   
14      B077TQN1BC       ONEPLUS   
15      B01H3V07EW       ONEPLUS   
16      B06XDXY25L       ONEPLUS   
17      B015FZLA8A       ONEPLUS   
18      B075QNGHS8         APPLE   
19      B077TQN1BC       ONEPLUS   
20      B00NQHZ0EA         APPLE   
21      B00VE1E0CI         APPLE   
22      B06XDXY25L       ONEPLUS   
23      B07K8K99TD         APPLE   
24      B01H3V07EW       ONEPLUS   
25      B07K8K99TD         APPLE   
26      B07HCRG2VD         A

In [44]:
pd.isnull(shuffled_df.values).any()

False

Cleaning the Text

In [49]:

def cleanTxt(t1):
    
    t1 = re.sub('[^ a-zA-Z]','',str(t1))
    t1 = re.sub(r' +', ' ', t1)
    return t1

lemma = WordNetLemmatizer()
def lemmatizeSentences(review_texts):
    count = 0
    for t1 in review_texts:
        
        t1 = cleanTxt(t1)
        sent = ""
        for i, j in pos_tag(word_tokenize(t1)):
            word = ''
            if j[0].lower() in ['n', 'v', 'r']:
                word = lemma.lemmatize(i, j[0].lower())
            elif j[0].lower() is 'j':
                word = lemma.lemmatize(i, 'a')
            else:
                word = lemma.lemmatize(i)
            sent = sent + " " + word.lower()
        review_texts[count] = sent
        count = count + 1;

    return review_texts

In [50]:
shuffled_df['review_texts'] = lemmatizeSentences(shuffled_df['review_texts'])

shuffled_df['review_texts'].head()

0     this phone be not unlocked a described att wo...
1                     sim card be bad can not activate
2                       excelent phone very good price
3     exactly a advertise from wireless expert inc ...
4     i love my t it be one of the best phone i hav...
Name: review_texts, dtype: object

In [51]:
# setting limit to display maximum columns for viewing results after creating DTM and printing
pd.set_option('display.max_columns', 1000)

**Constructing DTM using scikit-learn and then feature evaluation and normalizing**


In [85]:
tf_1 = CountVectorizer(min_df = 0.02 , max_df= 0.98,stop_words="english")

dtm_1 = pd.DataFrame(tf_1.fit_transform(shuffled_df['review_texts']).toarray(),columns=tf_1.get_feature_names())
dtm_1.head()

# using TfidfVectorizer to generate the weighted counts of vocabulary

tfidf_1 = TfidfVectorizer(min_df = 0.02 , max_df= 0.98, stop_words='english')
dtm_idf_1 = pd.DataFrame(tfidf_1.fit_transform(shuffled_df['review_texts']).toarray(),columns=tfidf_1.get_feature_names())
dtm_idf_1.head()

,able,absolutely,activate,allow,amazing,amazon,android,app,apple,apps,arrive,ask,att,away,awesome,bad,battery,best,better,big,box,brand,break,button,buy,cable,camera,card,carrier,carry,case,charge,charger,check,close,come,company,completely,condition,consider,contact,cost,customer,dash,data,day,deal,decide,design,device,didnt,different,display,doesnt,dont,drop,easily,easy,end,excellent,expect,expensive,experience,far,fast,feature,feel,fine,fix,gb,good,great,happy,headphone,help,high,home,hour,id,im,include,iphone,isnt,issue,item,ive,jack,just,know,le,life,like,line,list,little,lock,long,look,lot,love,make,market,maybe,miss,model,money,month,need,new,nice,number,old,oneplus,open,option,order,original,os,overall,oxygen,pay,people,perfect,perfectly,performance,phone,picture,plug,plus,point,power,pretty,price,problem,product,protector,purchase,quality,quickly,ram,really,reason,receive,recommend,return,review,right,run,samsung,say,screen,se,sell,seller,send,service,set,ship,sim,size,slow,small,software,sound,speaker,speed,star,start,stop,store,support,sure,switch,talk,tell,thats,thing,think,time,touch,try,turn,unless,unlock,unlocked,update,upgrade,use,user,version,video,wait,want,warranty,wasnt,way,website,week,wont,work,worth,year
0,0.0,0.0,0.608589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.522148,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.208450,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.559937,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.552143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.540411,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.453239,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.444599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.619107,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.337650,0.000000,0.0,0.0,0.0,0.0,0.0,0.709012,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [86]:
# creating tf and idf training and test samples

X_train, X_test, y_train, y_test = train_test_split(dtm_1,shuffled_df['product_lines'],test_size=0.30)
X_train_idf, X_test_idf, y_train_idf, y_test_idf = train_test_split(dtm_idf_1,shuffled_df['product_lines'],test_size=0.30)

In [87]:
#Logistic Regression Model
model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

#model.fit(X_train,y_train)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7811550151975684

In [88]:
model.score(X_test,y_test)

0.7811550151975684

In [89]:
cross_val_score(model,dtm_1,shuffled_df['product_lines'],cv=2).mean()

0.782034114273167

In [90]:
# Naive Bayes :Train the classifier on training data set

clf_NB = MultinomialNB()
clf_NB.fit(X_train,y_train)

# Prediction accuracy on test dataset
y_pred = clf_NB.predict(X_test)
clf_NB.score(X_test,y_test)

0.776595744680851

In [91]:
# Train the classifier on idf data set

clf_NB.fit(X_train_idf,y_train_idf)
y_pref_idf = clf_NB.predict(X_test_idf)

# Prediction accuracy
clf_NB.score(X_test_idf,y_test_idf)

0.8161094224924013

Results: both the models Naive Bayes and Logistice gave approximately similar results 

**Topic Modeling- topic scores for each of the document as features for classification**

In [92]:
lda = LatentDirichletAllocation(n_components=2)

dtm_lda = pd.DataFrame(lda.fit_transform(dtm_1))
dtm_lda.head()

,0,1
0,0.896994,0.103006
1,0.873134,0.126866
2,0.863914,0.136086
3,0.426139,0.573861
4,0.887612,0.112388


In [98]:
X_train, X_test, y_train, y_test = train_test_split(dtm_lda, shuffled_df['product_lines'],test_size=0.30)
model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

model.fit(X_train,y_train)
model.score(X_test,y_test)

0.6063829787234043